# Advent of Code - Day 7 Year 2024

In [1]:
DAY = 7
YEAR = 2024
#######
import json
import sys 
sys.path.append('../../')
from utils import get_pb_data, rephrase_pb, open_link, render_json
link, data, exmp, ae1, ae2, exmps, out = get_pb_data(DAY, YEAR)

print(f'problem link: {link}')

nb examples: 1
problem link: https://adventofcode.com/2024/day/7


In [2]:
# with open("./_puzzle_data.json", "r") as f: inp = json.loads(f.read())
# render_json(inp)

In [3]:
await rephrase_pb(DAY, YEAR, 1)

**Problem**: Given equations with missing operators, determine which ones can be made true using only + and * operators (evaluated left-to-right), and sum their test values. 

**Input format**: Each line contains:
- A test value (before the colon)
- A sequence of numbers (after the colon)
Example:
```
190: 10 19
3267: 81 40 27
83: 17 5
```

**Rules**:
- Only + and * operators can be used 
- Operators are evaluated left-to-right (no precedence) 
- Numbers must stay in their given order 
- Each space between numbers must be filled with one operator 

**Goal**: Find the sum of test values from equations that can be made true with some combination of operators 

**Example solution**: Only 3 equations are valid:
- 190 = 10 * 19
- 3267 = 81 + 40 * 27 (or 81 * 40 + 27)
- 292 = 11 + 6 * 16 + 20
Total = 190 + 3267 + 292 = 3749

In [4]:
(81+40)*27

3267

In [6]:
exmp

'190: 10 19\n3267: 81 40 27\n83: 17 5\n156: 15 6\n7290: 6 8 6 15\n161011: 16 10 13\n192: 17 8 14\n21037: 9 7 18 13\n292: 11 6 16 20'

How many different number of combinations exist ?

if I have :
- n numbers
- 2 operators

if n = 2, 1 different combinations

if n = 3, 4 different combinations

for n, 2**(n-1) combinations, and (n-1) operators

Let's parse the input as a dict: {test_val: list[nbs]}

In [24]:
lines = exmp.splitlines()
lines

['190: 10 19',
 '3267: 81 40 27',
 '83: 17 5',
 '156: 15 6',
 '7290: 6 8 6 15',
 '161011: 16 10 13',
 '192: 17 8 14',
 '21037: 9 7 18 13',
 '292: 11 6 16 20']

In [25]:
inp = {int(l.split(':')[0]): [int(n) for n in l.split(':')[1].strip().split(' ')]  for l in lines}

let's calculate the total number of possible combinations

In [27]:
sum([2**(len(v)-1) for v in inp.values()])

42

brute forcing will be fine.

I have the feeling that in part 2, the numbers on the right will represent intervals, and brute forcing will be much harder...

Let's build a function that given a test value and some input values, return all possible combinations

In [17]:
def get_combinations(tv: int, vals: list[int]) -> list[int]:
    ...

In [29]:
OPERATORS = ['+', '*']

In [19]:
s = '4*2'

claude: how do I evaluate a string representing an operation  in python already ?

In [21]:
tv = 3267
vals = [81, 40, 27]

In [28]:
nb_ops = len(vals)-1
nb_ops

2

In [30]:
def get_operators_comb(nb_vals: int) -> list[list[str]]:
    ...


claude informs me of the existence of itertools.product

In [32]:
from itertools import product

list(product(OPERATORS, repeat=3))

[('+', '+', '+'),
 ('+', '+', '*'),
 ('+', '*', '+'),
 ('+', '*', '*'),
 ('*', '+', '+'),
 ('*', '+', '*'),
 ('*', '*', '+'),
 ('*', '*', '*')]

In [33]:
def get_operators_comb(nb_vals: int) -> list[list[str]]:
    nb_ops = nb_vals - 1
    return list(product(OPERATORS, repeat=nb_ops))


In [34]:
nb_vals = 4
get_operators_comb(nb_vals)

[('+', '+', '+'),
 ('+', '+', '*'),
 ('+', '*', '+'),
 ('+', '*', '*'),
 ('*', '+', '+'),
 ('*', '+', '*'),
 ('*', '*', '+'),
 ('*', '*', '*')]

matchNow how do I apply those operations one step at a time ?

I remember functools has a function reduce that is handy ?

Actually no need, I can start iterating from index 1 on values, and index 0 on operators:

In [35]:
tv = 3267
vals = [81, 40, 27]
cbs = get_operators_comb(len(vals))
cbs

[('+', '+'), ('+', '*'), ('*', '+'), ('*', '*')]

In [36]:
for i in range(len(cbs)):
    print(cbs[i])

('+', '+')
('+', '*')
('*', '+')
('*', '*')


In [48]:
cb1 = cbs[3]
e = ''
for i in range(len(cb1)):
    if len(e) == 0:
        e = f"({vals[i]}{cb1[i]}{vals[i+1]})"
    else:
        e = f"({e}{cb1[i]}{vals[i+1]})" 
e

'((81*40)*27)'

In [50]:
possib = []
for cb in cbs:
    e = ''
    for i in range(len(cb)):
        if len(e) == 0:
            e = f"({vals[i]}{cb[i]}{vals[i+1]})"
        else:
            e = f"({e}{cb[i]}{vals[i+1]})" 
    possib.append(e)
possib

['((81+40)+27)', '((81+40)*27)', '((81*40)+27)', '((81*40)*27)']

In [51]:
def generate_possib(vals: list[int]) -> str:
    ...

In [52]:
def generate_possib(vals: list[int]) -> str:
    vals = [81, 40, 27]
    cbs = get_operators_comb(len(vals))
    possib = []
    for cb in cbs:
        e = ''
        for i in range(len(cb)):
            if len(e) == 0:
                e = f"({vals[i]}{cb[i]}{vals[i+1]})"
            else:
                e = f"({e}{cb[i]}{vals[i+1]})" 
        possib.append(e)
    return possib

In [56]:
dat = exmp 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
inp

[(190, [10, 19]),
 (3267, [81, 40, 27]),
 (83, [17, 5]),
 (156, [15, 6]),
 (7290, [6, 8, 6, 15]),
 (161011, [16, 10, 13]),
 (192, [17, 8, 14]),
 (21037, [9, 7, 18, 13]),
 (292, [11, 6, 16, 20])]

In [67]:
i = 1
tv = inp[i][0]
vals = inp[i][1]
print(inp[i])

(3267, [81, 40, 27])


In [62]:
generate_possib(vals)

['((81+40)+27)', '((81+40)*27)', '((81*40)+27)', '((81*40)*27)']

In [68]:
possib = [eval(s) for s in generate_possib(vals)]
tv in possib

True

In [69]:
def is_possible(tv: int, vals: list[int]) -> bool:
    possib = [eval(s) for s in generate_possib(vals)]
    return tv in possib

In [71]:
i = 0
tv = inp[i][0]
vals = inp[i][1]
print(inp[i])
is_possible(tv, vals)

(190, [10, 19])


False

Hmm. should be true

In [75]:
## forgot to remove vals here
def generate_possib(vals: list[int]) -> str:
    cbs = get_operators_comb(len(vals))
    possib = []
    for cb in cbs:
        e = ''
        for i in range(len(cb)):
            if len(e) == 0:
                e = f"({vals[i]}{cb[i]}{vals[i+1]})"
            else:
                e = f"({e}{cb[i]}{vals[i+1]})" 
        possib.append(e)
    return possib

In [79]:
def is_possible(tv: int, vals: list[int]) -> bool:
    possib = [eval(s) for s in generate_possib(vals)]
    return tv in possib
is_possible(tv, vals)

True

In [80]:
[tv for (tv, vals) in inp if is_possible(tv, vals)]

[190, 3267, 292]

In [81]:
sum([tv for (tv, vals) in inp if is_possible(tv, vals)])

3749

In [83]:
def sol1(dat: str) -> int:
    lines = dat.splitlines()
    inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
    return sum([tv for (tv, vals) in inp if is_possible(tv, vals)])
    

In [84]:
dat = exmp
sol1(dat)

3749

In [85]:
dat = data
sol1(dat)

303766880536

## part 2

In [86]:
await rephrase_pb(DAY, YEAR, 2)

**Problem**: Calculate the sum of all possible values that can be obtained by combining numbers with operators (+, *, ||), where:
- **||** is a concatenation operator that joins numbers (e.g., 12 || 345 = 12345)
- Operators are evaluated **left-to-right**
- For each line, find if it's possible to insert operators to match the target value
 
**Input example**:
```
156: 15 6
7290: 6 8 6 15
192: 17 8 14
```
 
For instance:
- 15 || 6 = 156
- 6 * 8 || 6 * 15 = 7290
- 17 || 8 + 14 = 192
 
Task is to find all valid equations and sum their target values.

We could brute force this again, but maybe it becomes too computationally expensive (?) let's try

In [93]:
dat = exmp
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]

In [94]:
## concatenation operator is same as an empty string linking both values
OPERATORS = ['+', '*', '']

In [95]:
def get_operators_comb(nb_vals: int) -> list[list[str]]:
    nb_ops = nb_vals - 1
    return list(product(OPERATORS, repeat=nb_ops))

In [97]:
i = 3
tv, vals = inp[i]
print(tv, vals)

156 [15, 6]


In [98]:
get_operators_comb(len(vals))

[('+',), ('*',), ('',)]

In [99]:
## forgot to remove vals here
def generate_possib(vals: list[int]) -> str:
    cbs = get_operators_comb(len(vals))
    possib = []
    for cb in cbs:
        e = ''
        for i in range(len(cb)):
            if len(e) == 0:
                e = f"({vals[i]}{cb[i]}{vals[i+1]})"
            else:
                e = f"({e}{cb[i]}{vals[i+1]})" 
        possib.append(e)
    return possib

In [100]:
generate_possib(vals)

['(15+6)', '(15*6)', '(156)']

Hm this doesn't work for 3 or more numbers

In [101]:
i = 1
tv, vals = inp[i]
print(tv, vals)

3267 [81, 40, 27]


In [102]:
generate_possib(vals)

['((81+40)+27)',
 '((81+40)*27)',
 '((81+40)27)',
 '((81*40)+27)',
 '((81*40)*27)',
 '((81*40)27)',
 '((8140)+27)',
 '((8140)*27)',
 '((8140)27)']

In [103]:
eval('(8140)27')

SyntaxError: invalid syntax (<string>, line 1)

But it does work if I evaluate the result after each step

In [112]:
## forgot to remove vals here
def generate_possib(vals: list[int]) -> str:
    cbs = get_operators_comb(len(vals))
    possib = []
    for cb in cbs:
        e = ''
        for i in range(len(cb)):
            if len(e) == 0:
                e = f"({vals[i]}{cb[i]}{vals[i+1]})"
            else:
                e = f"({e}{cb[i]}{vals[i+1]})"
            e = str(eval(e))
        possib.append(e)
    return possib

In [113]:
vals

[81, 40, 27]

In [114]:
generate_possib(vals)

['148', '3267', '12127', '3267', '87480', '324027', '8167', '219780', '814027']

In [107]:
def is_possible(tv: int, vals: list[int]) -> bool:
    return str(tv) in generate_possib(vals)

In [110]:
i = 1
tv, vals = inp[i]
print(tv, vals)
is_possible(tv, vals)

3267 [81, 40, 27]


True

In [116]:
dat = exmp 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
for ip in inp:
    tv, vals = ip
    print(tv, vals)
    print(is_possible(tv, vals))
    print('---')

190 [10, 19]
True
---
3267 [81, 40, 27]
True
---
83 [17, 5]
False
---
156 [15, 6]
True
---
7290 [6, 8, 6, 15]
True
---
161011 [16, 10, 13]
False
---
192 [17, 8, 14]
True
---
21037 [9, 7, 18, 13]
False
---
292 [11, 6, 16, 20]
True
---


In [118]:
dat = exmp 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
sum([tv for (tv, vals) in inp if is_possible(tv, vals)])

11387

In [122]:
from tqdm import tqdm
def sol2(dat: str) -> int:
    lines = dat.splitlines()
    inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
    return sum([tv for (tv, vals) in tqdm(inp) if is_possible(tv, vals)])
    

In [123]:
dat = data
sol2(dat)

 19%|█▉        | 160/850 [03:04<13:14,  1.15s/it]


KeyboardInterrupt: 

brute force is possible but a bit slow any other option ?

I could STOP EARLY: if the current eval is greater than the test value, it is impossible and we should stop (except if there are zeros ?)

In [125]:
dat = data 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]

In [126]:
len(inp)

850

In [130]:
min([v for tv, vals in inp for v in vals])

1

the minimum value is 1, so I should stop as soon as the current evaluation is above the test value

In [133]:
def is_comb_possible(cb: list[str], tv: int, vals: list[int]) -> bool:
    ...

In [182]:
def is_comb_possible(cb: list[str], tv: int, vals: list[int]) -> bool:
    e = ''
    for i in range(len(cb)):
        if len(e) == 0:
            e = f"({vals[i]}{cb[i]}{vals[i+1]})"
        else:
            e = f"({e}{cb[i]}{vals[i+1]})"
        #print('e before eval:', e)
        e = eval(e)
        #print('e after eval:', e)
        if e > tv:
            return False
        e = str(e)
    return tv == int(e)

In [183]:
cb = ['*', '']
vals = [17, 90, 14]
tv = 192
is_comb_possible(cb, tv, vals)

False

In [184]:
dat = data 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]

In [185]:
i = 1
tv, vals = inp[i]
cbs = get_operators_comb(len(vals))
print(tv, vals)
print(len(cbs))
for cb in cbs:
    print(is_comb_possible(cb, tv, vals))


2201728328 [63, 5, 8, 511, 10, 2, 8, 99, 7]
6561
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fal

In [186]:
# let's check if there are cases where one element in the list of values is superior to the test value

In [188]:
dat = data 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
any([max(vals) > tv for tv, vals in inp])

False

In [189]:
def is_possible_2(tv: int, vals: list[int]) -> bool:
    cbs = get_operators_comb(len(vals))
    for cb in cbs:
        if is_comb_possible(cb, tv, vals):
            return True
    return False 

In [194]:
i = 1
tv, vals = inp[i]
print(tv, vals)
print(len(cbs))
print(is_possible_2(tv, vals))

2201728328 [63, 5, 8, 511, 10, 2, 8, 99, 7]
6561
False


In [197]:
dat = exmp 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
sum([tv for tv, vals in tqdm(inp) if is_possible_2(tv, vals)])

100%|██████████| 9/9 [00:00<00:00, 5733.40it/s]


11387

In [198]:
dat = data 
lines = dat.splitlines()
inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
sum([tv for tv, vals in tqdm(inp) if is_possible_2(tv, vals)])

100%|██████████| 850/850 [07:38<00:00,  1.85it/s]


337041851384440

Not much faster...

What could speed things up ?

I'm creating strings at every iteration ...

I am calling eval every time I add an operator, which is a lot

## improving the solution

In [314]:
import math 
def add(x,y):
    return x+y 
def mult(x, y):
    return x*y
def count_digits(n):
    if n == 0:
        return 1
    return math.floor(math.log10(abs(n))) + 1
def concat(x, y):
    return x*10**count_digits(y) + y
def create_op_gen(op_comb): 
    yield from op_comb

In [372]:
OPERATORS = [add, mult, concat]
vals = [1,2,3]
all_combs = product(OPERATORS, repeat=len(vals)-1)

In [380]:
op_comb = next(all_combs)
op_gen = create_op_gen(op_comb)

In [381]:
from functools import reduce
reduce(lambda x, y: next(op_gen)(x,y), vals)

6

In [385]:
vals = [1,2,3]
all_ops_combinations = list(product(OPERATORS, repeat=len(vals)-1))
all_ops_gen = [create_op_gen(op_comb) for op_comb in all_ops_combinations]
[reduce(lambda x, y: next(op_gen)(x,y), vals) for op_gen in all_ops_gen]

[6, 9, 33, 5, 6, 23, 15, 36, 123]

In [387]:
def is_possible(tv: int, vals: list[int]) -> bool:
    all_ops_combinations = list(product(OPERATORS, repeat=len(vals)-1))
    all_ops_gen = [create_op_gen(op_comb) for op_comb in all_ops_combinations]
    return tv in [reduce(lambda x, y: next(op_gen)(x,y), vals) for op_gen in all_ops_gen]

In [400]:
def format_input(dat: str):
    lines = dat.splitlines()
    inp = [(int(l.split(':')[0]), [int(n) for n in l.split(':')[1].strip().split(' ')]) for l in lines]
    return inp

In [390]:
dat = exmp
inp = format_input(dat)


190 [10, 19]


In [393]:
for i in range(len(inp)):
    tv, vals = inp[i]
    print(tv, vals)
    print(is_possible(tv, vals))

190 [10, 19]
True
3267 [81, 40, 27]
True
83 [17, 5]
False
156 [15, 6]
True
7290 [6, 8, 6, 15]
True
161011 [16, 10, 13]
False
192 [17, 8, 14]
True
21037 [9, 7, 18, 13]
False
292 [11, 6, 16, 20]
True


In [395]:
sum([tv for tv, vals in inp if is_possible(tv, vals)])

11387

In [403]:
def sol2(dat: str) -> int:
    inp = format_input(dat)
    return sum([tv for tv, vals in tqdm(inp) if is_possible(tv, vals)])
    

In [404]:
dat = exmp 
sol2(dat)

100%|██████████| 9/9 [00:00<00:00, 32486.00it/s]


11387

In [405]:
dat = data 
sol2(dat)

100%|██████████| 850/850 [01:01<00:00, 13.72it/s]


337041851384440

Below is solution translated to python from clojure code solution found by a colleague:

In [415]:
def parse_input(filename):
    with open(filename) as f:
        lines = f.readlines()
    
    parsed_data = []
    for line in lines:
        # Split on ": " to separate value and numbers
        value_str, nums_str = line.strip().split(": ")
        # Create a dictionary (Python's equivalent of Clojure map)
        parsed_data.append({
            'value': int(value_str),
            'nums': [int(n) for n in nums_str.split()]
        })
    return parsed_data

def concatenate(a, b):
    return int(str(a) + str(b))

def calibrates(data, computed_so_far):
    value = data['value']
    nums = data['nums']
    
    # Base cases (direct translation of Clojure cond)
    if not nums and value == computed_so_far:
        return value
    if not nums or computed_so_far > value:
        return 0
        
    # Create next_value_nums (equivalent to Clojure's rest)
    next_value_nums = {
        'value': value,
        'nums': nums[1:]  # Python's equivalent of rest
    }
    
    # Try all three operations
    first_num = nums[0]
    mult = calibrates(next_value_nums, 
                     first_num if computed_so_far == 0 else computed_so_far * first_num)
    add = calibrates(next_value_nums, 
                    computed_so_far + first_num)
    concat = calibrates(next_value_nums, 
                       concatenate(computed_so_far, first_num))
    
    # If any operation returns positive, return value
    if mult > 0 or add > 0 or concat > 0:
        return value
    return 0

def solve(parsed_data):
    # Map calibrates over parsed data and sum results
    return sum(calibrates(data, 0) for data in parsed_data)

In [414]:
dat = data
inp = format_input(dat)
fmt = [{"value":tv, "nums": nums} for tv, nums in inp]
solve(fmt)

337041851384440

## takeaway:

- This problem focuses on **recursion**. Using recursion enables early stoppage and make the code much more efficient

- Find the problem solving process to tackle this step by step

[]